# Reproduce DL
## Automated Pavement Crack Segmentation

We start by setting up the actual Architecture. This means making sure all weights are properly initialized and all layers are connected. 

We make use of PyTorch for the implementation.

Multiple parts come together (A U-based ResNet);
- We recreate ResNet34 and remove the last two layers
- We made sure that a ResNet-block is either 4 or 6 layers depending on if stride is not 1 (which in our case always happens when the in_channels are not equal to out_channels)
- We use transfer learning such that the ResNet34 parameters are initialized as if trained on ImageNet
- We create Squeeze and Excitation blocks that are applied per Channel (cSE) and per Spatial (sSE) (image)
- These two blocks are combined (scSE) and then the maximum of this is taken
- Each convolutional layer its parameters are initialized via "He Kaiming" method.

In [4]:
from architecture import main
from torchsummary import summary

network = main.Net()
# print(network)
summary(network, input_size=(3, 320, 480))

            Conv2d-1         [-1, 64, 160, 240]           9,408
            Conv2d-2         [-1, 64, 160, 240]           9,408
       BatchNorm2d-3         [-1, 64, 160, 240]             128
       BatchNorm2d-4         [-1, 64, 160, 240]             128
              ReLU-5         [-1, 64, 160, 240]               0
              ReLU-6         [-1, 64, 160, 240]               0
            Conv2d-7        [-1, 128, 160, 240]           8,320
         MaxPool2d-8          [-1, 64, 80, 120]               0
         MaxPool2d-9          [-1, 64, 80, 120]               0
           Conv2d-10          [-1, 64, 80, 120]          36,864
           Conv2d-11          [-1, 64, 80, 120]          36,864
      BatchNorm2d-12          [-1, 64, 80, 120]             128
      BatchNorm2d-13          [-1, 64, 80, 120]             128
             ReLU-14          [-1, 64, 80, 120]               0
             ReLU-15          [-1, 64, 80, 120]               0
           Conv2d-16          [-1, 64, 8

In [5]:
import os
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from datasets.CFD.CFDdata import CFD

current_path = os.path.abspath(os.getcwd())

totensor = transforms.ToTensor()
dataset = CFD(current_path + "/datasets/CFD/cfd_image/", totensor, current_path + "/datasets/CFD/seg_gt/", totensor)
gen = torch.Generator().manual_seed(42)

train_data, test_data = random_split(dataset, [71, 47], gen)
train_loader = DataLoader(train_data, batch_size=10)
test_loader = DataLoader(test_data, batch_size=47)

print("The current dataset is size: ", len(dataset), 
"\nThe current split is train/test: ", len(train_loader.dataset), "/", len(test_loader.dataset))


# fig, axs = plt.subplots(5, 5, figsize=(5, 5))
# for i in range(25):
#     _, gt = test_data[i]
#     ax = axs[i // 5][i % 5]
#     ax.imshow(gt.view(320, 480), cmap="gray")
#     ax.axis("off")
# plt.tight_layout()
# plt.show()

The current dataset is size:  118 
The current split is train/test:  71 / 47


In [6]:
# Some functions used during training

def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device


def evaluate_accuracy(data_loader, network, device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    network.eval()  #make sure network is in evaluation mode

    #init
    acc_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(network(X), dim=1) == y))
            n += y.shape[0] #increases with the number of samples in the batch
    return acc_sum.item()/n


In [7]:
from loss import dice_loss

# training parameters
lr = 1e-3
epochs = 1

#Initialize network
network = main.Net()
optimizer = torch.optim.AdamW(network.parameters(), lr=lr, betas=(0.9, 0.999))
criterion = dice_loss.batch_dice_loss

# Define list to store losses and performances of each interation
train_losses = []
train_accs = []
test_accs = []

# Try using gpu instead of cpu
device = try_gpu()

for epoch in range(epochs):

    # Network in training mode and to device
    network.train()
    network.to(device)

    # Training loop
    for i, (x_batch, y_batch) in enumerate(train_loader):

         # Set to same device
         x_batch, y_batch = x_batch.to(device), y_batch.to(device)

         # Set the gradients to zero
         optimizer.zero_grad()

         # Perform forward pass
         y_pred = network(x_batch)

         # Compute the loss
         loss = criterion(y_pred, y_batch)
         train_losses.append(loss)

         # Backward computation and update
         loss.backward()
         optimizer.step()
    
    # Compute train and test error
    train_acc = 100*evaluate_accuracy(train_loader, network.to('cpu'))
    test_acc = 100*evaluate_accuracy(test_loader, network.to('cpu'))
    
    # Development of performance
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    # Print performance
    print('Epoch: {:.0f}'.format(epoch+1))
    print('Accuracy of train set: {:.00f}%'.format(train_acc))
    print('Accuracy of test set: {:.00f}%'.format(test_acc))
    print('')


Epoch: 1
Accuracy of train set: 148975763%
Accuracy of test set: 711764289%



In [ ]:
# from torchviz import make_dot

# x = torch.randn(1,3, 320, 480)
# y = net(x) 

# make_dot(y).view()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=411d58e9-cb4b-4924-bef0-2f383eff0187' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>